# CIS 583 Term Project – OrderBot with Two LLM Architectures

This notebook implements a pizza-ordering chatbot (OrderBot) using **two local GGUF LLMs**
via `llama-cpp-python`. It is designed so you can compare the behavior of two different
architectures (e.g., Llama 2 and Mistral).

In [ ]:
%%capture
!pip install llama-cpp-python huggingface_hub panel jupyter_bokeh pandas

In [ ]:
import os
from huggingface_hub import hf_hub_download

os.makedirs('models', exist_ok=True)

MODEL_A_REPO_ID = 'TheBloke/Llama-2-7B-Chat-GGUF'
MODEL_A_FILENAME = 'llama-2-7b-chat.Q4_K_M.gguf'

MODEL_B_REPO_ID = 'TheBloke/Mistral-7B-Instruct-v0.2-GGUF'
MODEL_B_FILENAME = 'mistral-7b-instruct-v0.2.Q4_K_M.gguf'

def download_model(repo_id, filename):
    local_path = os.path.join('models', filename)
    if os.path.exists(local_path):
        print(' Using existing local model:', local_path)
        return local_path

    print(f' Downloading {repo_id}/{filename} ...')
    path = hf_hub_download(
        repo_id=repo_id,
        filename=filename,
        local_dir='models',
        local_dir_use_symlinks=False,
    )
    print(' Downloaded to:', path)
    return path

model_a_path = download_model(MODEL_A_REPO_ID, MODEL_A_FILENAME)
model_b_path = download_model(MODEL_B_REPO_ID, MODEL_B_FILENAME)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:979: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


llama-2-7b-chat.Q4_K_M.gguf:   0%|          | 0.00/4.08G [00:00<?, ?B/s]

 Downloaded to: models/llama-2-7b-chat.Q4_K_M.gguf


mistral-7b-instruct-v0.2.Q4_K_M.gguf:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

 Downloaded to: models/mistral-7b-instruct-v0.2.Q4_K_M.gguf


In [ ]:
from llama_cpp import Llama

class LlamaChat:
    def __init__(self, model_path: str, name: str):
        self.name = name
        self.llm = Llama(
            model_path=model_path,
            n_ctx=4096,
            n_gpu_layers=0,
            verbose=False,
        )

    def chat(self, messages):
        """messages is a list of {role, content} dicts (OpenAI-style)."""
        result = self.llm.create_chat_completion(messages=messages)
        return result['choices'][0]['message']['content']

llm_a = LlamaChat(model_a_path, name='Llama-2-7B-Chat')
llm_b = LlamaChat(model_b_path, name='Mistral-7B-Instruct-v0.2')

print('Backends ready:', llm_a.name, 'and', llm_b.name)


llama_context: n_ctx_per_seq (4096) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


Backends ready: Llama-2-7B-Chat and Mistral-7B-Instruct-v0.2


In [ ]:
SYSTEM_PROMPT = '''\
You are OrderBot, an automated service for XYZ Pizza.

Your goals:
1. Greet the customer politely.
2. Ask for the customer's name.
3. Collect the full pizza order:
   - pizza type
   - size
   - quantity
   - extras/toppings
4. Ask if the order is pickup or delivery.
   - If delivery: ask for address and phone number.
5. Confirm the complete order back to the customer.
6. Provide an itemized price, tax, and total.
7. Ask if they want to change anything before finalizing.

Stay focused on the pizza order. Be friendly and concise.
'''

def make_initial_context():
    return [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "assistant", "content": "Hello! Welcome to XYZ Pizza. What is your name?"},
    ]


In [ ]:
test_scenarios = [
    {
        'name': 'simple_one_pizza',
        'user_turns': [
            'Hi, my name is Osamah.',
            'I want one large pepperoni pizza.',
            'Delivery please.',
            'My address is 123 Main Street and my phone is 555-0000.',
            'Yes, that is everything.'
        ],
        'expected_slots': {
            'name': 'Osamah',
            'items': [('pepperoni pizza', 'large', 1)],
            'delivery_or_pickup': 'delivery',
        },
    },
    {
        'name': 'multiple_pizzas',
        'user_turns': [
            "Hello, I'm Sara.",
            'I want two medium cheese pizzas and one small veggie pizza.',
            'Pickup.',
            'Yes, that is correct.'
        ],
        'expected_slots': {
            'name': 'Sara',
            'items': [
                ('cheese pizza', 'medium', 2),
                ('veggie pizza', 'small', 1),
            ],
            'delivery_or_pickup': 'pickup',
        },
    },

]


In [ ]:
def run_scenario(llm: LlamaChat, scenario):
    messages = make_initial_context()
    transcript = []

    for user_msg in scenario['user_turns']:
        messages.append({'role': 'user', 'content': user_msg})
        assistant_reply = llm.chat(messages)
        messages.append({'role': 'assistant', 'content': assistant_reply})

        transcript.append({
            'user': user_msg,
            'assistant': assistant_reply,
        })

    return {
        'model': llm.name,
        'scenario': scenario['name'],
        'transcript': transcript,
    }


In [ ]:
def score_transcript(result, scenario):
    text = ' '.join(t['assistant'] for t in result['transcript']).lower()
    score = 0
    max_score = 3

    if scenario['expected_slots']['name'].lower() in text:
        score += 1

    all_items_ok = True
    for (ptype, size, qty) in scenario['expected_slots']['items']:
        if ptype.lower() not in text or size.lower() not in text:
            all_items_ok = False
            break
    if all_items_ok:
        score += 1

    mode = scenario['expected_slots']['delivery_or_pickup'].lower()
    if mode in text:
        score += 1

    return score, max_score


In [ ]:
import pandas as pd

all_results = []

for scenario in test_scenarios:
    # Model A
    res_a = run_scenario(llm_a, scenario)
    score_a, max_s = score_transcript(res_a, scenario)

    # Model B
    res_b = run_scenario(llm_b, scenario)
    score_b, _ = score_transcript(res_b, scenario)

    all_results.append({
        'scenario': scenario['name'],
        'model_a': llm_a.name,
        'model_b': llm_b.name,
        'score_a': score_a,
        'score_b': score_b,
        'max_score': max_s,
    })

df_results = pd.DataFrame(all_results)
df_results


,scenario,model_a,model_b,score_a,score_b,max_score
0,simple_one_pizza,Llama-2-7B-Chat,Mistral-7B-Instruct-v0.2,3,3,3
1,multiple_pizzas,Llama-2-7B-Chat,Mistral-7B-Instruct-v0.2,3,2,3


In [ ]:
# Example: print transcript for first scenario using Model A
example_res_a = run_scenario(llm_a, test_scenarios[0])
for turn in example_res_a['transcript']:
    print('USER:', turn['user'])
    print('BOT :', turn['assistant'])
    print('-' * 40)



USER: Hi, my name is Osamah.
BOT :   Great, Osamah! Thank you for ordering from XYZ Pizza. Can you please tell me what type of pizza you would like to order? We have a variety of options on our menu, such as Pepperoni, Margherita, Meat Lover's, and more. Please let me know the size and quantity you would like, as well as any extra toppings you would like to add.
----------------------------------------
USER: I want one large pepperoni pizza.
BOT :   Great choice, Osamah! One large Pepperoni Pizza coming right up. Would you like to pick up your order or have it delivered? If delivery, could you please provide me with your address and phone number?
----------------------------------------
USER: Delivery please.
BOT :   Great, thank you! I will confirm your order and provide you with an estimated delivery time. Your order will be delivered to your address within the estimated time frame. Is there anything else I can assist you with?
----------------------------------------
USER: My addres

In [ ]:
example_res_b = run_scenario(llm_b, test_scenarios[0])
for turn in example_res_b['transcript']:
    print("USER:", turn['user'])
    print("BOT :", turn['assistant'])
    print("-" * 40)



USER: Hi, my name is Osamah.
BOT :  Hi Osamah! It's nice to meet you. How can I help you today at XYZ Pizza? Would you like to order a pizza for delivery or pick-up? We have a variety of delicious pizzas to choose from, as well as sides and desserts. Let me know if you have any dietary restrictions or preferences, and I'll be happy to help you find something that suits your needs. If you're not quite ready to order yet, feel free to browse our menu on our website or ask me any questions you might have. I'm here to make your experience at XYZ Pizza as enjoyable as possible!
----------------------------------------
USER: I want one large pepperoni pizza.
BOT :  Great choice, Osamah! A large pepperoni pizza is one of our most popular options. I'll make sure to get that started for you right away. Would you like it for delivery or pick-up? And would you like any sides or extras with that? For example, we have garlic knots, wings, salads, and various toppings that you can add to your pizza 

In [ ]:
# =========================
# Interactive Chat Section
# =========================

def chat_with_model(llm, user_message):
    """One-shot chat: send a single message and get a reply."""
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_message}
    ]
    reply = llm.chat(messages)
    print(f"{llm.name} BOT:", reply)


def chat_loop(llm):
    """Continuous chat loop. Type 'exit' to stop."""
    messages = [{"role": "system", "content": SYSTEM_PROMPT}]
    print(f"Start chatting with {llm.name}! Type 'exit' to stop.\n")

    while True:
        user = input("You: ")
        if user.lower().strip() in ["exit", "quit", "stop"]:
            print("Chat ended.")
            break
        messages.append({"role": "user", "content": user})
        bot = llm.chat(messages)
        messages.append({"role": "assistant", "content": bot})
        print(f"{llm.name} Bot:", bot, "\n")

# Examples:
# chat_with_model(llm_a, "Hi, I want to order pizza.")
# chat_with_model(llm_b, "Do you have vegetarian options?")
# chat_loop(llm_a)
# chat_loop(llm_b)


In [ ]:
chat_with_model(llm_a, "Hi, I want to order pizza.")


Llama-2-7B-Chat BOT:   Great! Welcome to XYZ Pizza. My name is OrderBot, and I'll be taking your order today. Can I please have your name, please?


In [ ]:
import os, json
from datetime import datetime

# Folder to store chat logs
LOG_DIR = "chat_logs"
os.makedirs(LOG_DIR, exist_ok=True)

def save_transcript(model_name, messages):
    """Save full chat messages to a timestamped txt + json file."""
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    base = f"{model_name}_{ts}"

    # Save as JSON (structured)
    json_path = os.path.join(LOG_DIR, base + ".json")
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(messages, f, indent=2)

    # Save as TXT (human readable)
    txt_path = os.path.join(LOG_DIR, base + ".txt")
    with open(txt_path, "w", encoding="utf-8") as f:
        for m in messages:
            f.write(f"{m['role'].upper()}: {m['content']}\n\n")

    print(f"✅ Transcript saved:\n- {txt_path}\n- {json_path}")


In [ ]:
def chat_loop_with_logging(llm):
    messages = [{"role": "system", "content": SYSTEM_PROMPT}]
    print(f"\n🟢 Chatting with {llm.name}. Type 'exit' to stop.\n")

    while True:
        user = input("You: ")
        if user.lower().strip() in ["exit", "quit", "stop"]:
            print("Chat ended.")
            break

        messages.append({"role": "user", "content": user})
        bot = llm.chat(messages)
        messages.append({"role": "assistant", "content": bot})
        print(f"{llm.name} Bot:", bot, "\n")

    # auto-save at end
    save_transcript(llm.name, messages)


def choose_model_and_chat():
    print("\nChoose a model to chat with:")
    print("1) Model A (Llama-2-7B-Chat)")
    print("2) Model B (Mistral-7B-Instruct v0.2)")

    choice = input("Enter 1 or 2: ").strip()

    if choice == "1":
        chat_loop_with_logging(llm_a)
    elif choice == "2":
        chat_loop_with_logging(llm_b)
    else:
        print("❌ Invalid choice. Run again and pick 1 or 2.")


In [ ]:
choose_model_and_chat()
